In [27]:
import pandas as pd
import numpy as np
from datetime import datetime, date
from pandas import Timestamp
import csv


In [15]:
custom_date_parser=lambda dates: [ datetime.strptime(str(d), "%Y-%m-%d") for d in dates]  
# for datetime "%Y-%m-%d %H:%M:%S"
dfraw = pd.read_csv('data.csv', quoting=csv.QUOTE_NONNUMERIC,quotechar='"',sep = ',', header = 0, index_col = None)
dfraw["date_trans"]=pd.to_datetime(df["date_trans"],format="%Y-%m-%d")
dfraw

,id,ref,date_trans,status,amount
0,1.0,INV-220001,2022-01-01,OPEN,35.94
1,2.0,INV-220002,2022-01-02,OPEN,99.99
2,3.0,INV-220003,2022-01-03,CANCELLED,13.00
3,4.0,INV-220003,2022-01-04,OPEN,NaN
4,5.0,INV-220005,2022-01-04,OPEN,204.20
5,6.0,INV-220006,2022-01-15,OPEN,-4.20
6,7.0,NaN,2022-01-06,NaN,0.00
7,8.0,INV-220007,2022-01-15,PENDING,50.40
8,9.0,NaN,NaT,ERROR,0.00
9,10.0,INV-220007,2022-01-15,PENDING,50.40


In [39]:
def dataframe_to_code(df):
    data = np.array2string(df.to_numpy(), separator=', ')
    data = data.replace(" nan", " float('nan')")
    data = data.replace(" NaT", " pd.NaT")

    cols = df.columns.tolist()
    return f"""df = pd.DataFrame({data}, columns={cols})"""
#print(dataframe_to_code(dfraw))
#use this to create the code for dataframes

df = pd.DataFrame([[1.0, 'INV-220001', Timestamp('2022-01-01 00:00:00'), 'OPEN', 35.94],
 [2.0, 'INV-220002', Timestamp('2022-01-02 00:00:00'), 'OPEN', 99.99],
 [3.0, 'INV-220003', Timestamp('2022-01-03 00:00:00'), 'CANCELLED', 13.0],
 [4.0, 'INV-220003', Timestamp('2022-01-04 00:00:00'), 'OPEN', float('nan')],
 [5.0, 'INV-220005', Timestamp('2022-01-04 00:00:00'), 'OPEN', 204.2],
 [6.0, 'INV-220006', Timestamp('2022-01-15 00:00:00'), 'OPEN', -4.2],
 [7.0, float('nan'), Timestamp('2022-01-06 00:00:00'), float('nan'), 0.0],
 [8.0, 'INV-220007', Timestamp('2022-01-15 00:00:00'), 'PENDING', 50.4],
 [9.0, float('nan'), pd.NaT, 'ERROR', 0.0],
 [10.0, 'INV-220007', Timestamp('2022-01-15 00:00:00'), 'PENDING', 50.4]], columns=['id', 'ref', 'date_trans', 'status', 'amount'])


In [11]:
df = pd.DataFrame([
 [1.0, 'INV-220001', Timestamp('2022-01-01 00:00:00'), 'OPEN', 35.94],
 [2.0, 'INV-220002', Timestamp('2022-01-02 00:00:00'), 'OPEN', 99.99],
 [3.0, 'INV-220003', Timestamp('2022-01-03 00:00:00'), 'CANCELLED', 13.0],
 [4.0, 'INV-220003', Timestamp('2022-01-04 00:00:00'), 'OPEN', float('nan')],
 [5.0, 'INV-220005', Timestamp('2022-01-04 00:00:00'), 'OPEN', 204.2],
 [6.0, 'INV-220006', Timestamp('2022-01-15 00:00:00'), 'OPEN', -4.2],
 [7.0, float('nan'), Timestamp('2022-01-06 00:00:00'), float('nan'), 0.0],
 [8.0, 'INV-220007', Timestamp('2022-01-15 00:00:00'), 'PENDING', 50.4],
 [9.0, '', pd.NaT, 'ERROR', 0.0],
 [10.0, 'INV-220007', Timestamp('2022-01-15 00:00:00'), 'PENDING', 50.4]], columns=['id', 'ref', 'date_trans', 'status', 'amount'])
df

,id,ref,date_trans,status,amount
0,1.0,INV-220001,2022-01-01,OPEN,35.94
1,2.0,INV-220002,2022-01-02,OPEN,99.99
2,3.0,INV-220003,2022-01-03,CANCELLED,13.00
3,4.0,INV-220003,2022-01-04,OPEN,NaN
4,5.0,INV-220005,2022-01-04,OPEN,204.20
5,6.0,INV-220006,2022-01-15,OPEN,-4.20
6,7.0,NaN,2022-01-06,NaN,0.00
7,8.0,INV-220007,2022-01-15,PENDING,50.40
8,9.0,,NaT,ERROR,0.00
9,10.0,INV-220007,2022-01-15,PENDING,50.40


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          10 non-null     float64       
 1   ref         9 non-null      object        
 2   date_trans  9 non-null      datetime64[ns]
 3   status      9 non-null      object        
 4   amount      9 non-null      float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 528.0+ bytes


In [12]:
from pandas.api.types import is_datetime64_any_dtype as is_datetime
def check(in_df):
    df=in_df.copy()
    dtypes = df.dtypes.to_dict()
    col_metrics=[]
    for col, typ in dtypes.items():
        metrics={}
        metrics["column"]=col
        metrics["dtype"]=typ
        metrics["records"]=len(df[col])
        metrics["count_unique"]=len(set(df[pd.notna(df[col])][col]))
        metrics["nans"]=len(df[pd.isnull(df[col])])

        
        if metrics["count_unique"]<5:
            value_counts_series=df[col].value_counts(dropna=False)
            metrics["value_counts"]=value_counts_series.to_dict()
        else:
            metrics["value_counts"]=[]
        if typ=="float" or typ=="int64" or is_datetime(df[col]):
            metrics["max"]=max(df[col])
            metrics["min"]=min(df[col])    
        elif typ=="object":
            values=df[pd.notna(df[col])][col]
            values_numeric_chars=values.str.replace(r'\D+', '')
            values_numeric_chars_no_blank= values_numeric_chars[values_numeric_chars.str.len()>0]
            values_numeric=pd.to_numeric(values_numeric_chars_no_blank, errors='coerce')
            if len(values_numeric)>0:
                metrics["max"]=max(values_numeric)
                metrics["min"]=min(values_numeric)
            metrics["empty_strings"]=len(df[df[col].eq('')])
            
        #print(metrics)
        col_metrics.append(metrics)
    return pd.DataFrame(col_metrics)     
        
check(df)

,column,dtype,records,count_unique,nans,value_counts,max,min,empty_strings
0,id,float64,10,10,0,[],10,1,NaN
1,ref,object,10,7,1,[],220007,220001,1.0
2,date_trans,datetime64[ns],10,6,1,[],2022-01-15 00:00:00,2022-01-01 00:00:00,NaN
3,status,object,10,4,1,"{'OPEN': 5, 'PENDING': 2, 'ERROR': 1, 'CANCELL...",NaN,NaN,0.0
4,amount,float64,10,7,1,[],204.2,-4.2,NaN
